### This file is used to extract the input/output positions and velocities, lane, and lane norm from the training and test data

In [1]:
import os, os.path 
import pickle
from glob import glob

import torch
import numpy as np
import pandas as pd

import tqdm
import tqdm.notebook

In [2]:
# File paths
TEST_PATH = './new_val_in'  # Path to input of the test set
TRAIN_PATH = './new_train'  # Path to input + output of training set

DUMMY_TRAIN_PATH = './dummy_train'
DUMMY_TEST_PATH = './dummy_val'

IN_POS_FILE = './input_positions.csv'
OUT_POS_FILE = './output_positions.csv'

IN_VEL_FILE = './input_velocities.csv'
OUT_VEL_FILE = './output_velocities.csv'

LANE_FILE = './lane.txt'
LANE_NORM_FILE = './lane_norm.txt'

IN_POS_FILE_TEST = './input_positions_test.csv'
IN_VEL_FILE_TEST = './input_velocities_test.csv'

LANE_FILE_TEST = './lane_test.txt'
LANE_NORM_FILE_TEST = './lane_norm_test.txt'

# Keys to the pickle objects
CITY = 'city'
LANE = 'lane'
LANE_NORM = 'lane_norm'
SCENE_IDX = 'scene_idx'
AGENT_ID = 'agent_id'
P_IN = 'p_in'
V_IN = 'v_in'
P_OUT = 'p_out'
V_OUT = 'v_out'
CAR_MASK = 'car_mask'
TRACK_ID = 'track_id'

# Additional keys for DataFrames
WAS_TARGET = 'was_target'
P_IN_X = ['p_in_x' + str(i) for i in range(1, 20)]
P_IN_Y = ['p_in_y' + str(i) for i in range(1, 20)]
V_IN_X = ['v_in_x' + str(i) for i in range(1, 20)]
V_IN_Y = ['v_in_y' + str(i) for i in range(1, 20)]

P_OUT_X = ['p_out_x' + str(i) for i in range(1, 20)]
P_OUT_Y = ['p_out_y' + str(i) for i in range(1, 20)]
V_OUT_X = ['v_out_x' + str(i) for i in range(1, 20)]
V_OUT_Y = ['v_out_y' + str(i) for i in range(1, 20)]

P_IN_CSV_HEADER = [SCENE_IDX, CITY, TRACK_ID, WAS_TARGET] + P_IN_X + P_IN_Y
# Add commas for each element
P_IN_CSV_HEADER = [col + ',' for col in P_IN_CSV_HEADER]
# Remove the last commas
P_IN_CSV_HEADER[-1] = P_IN_CSV_HEADER[-1].rstrip(',')

V_IN_CSV_HEADER = [SCENE_IDX, CITY, TRACK_ID, WAS_TARGET] + V_IN_X + V_IN_Y
# Add commas for each element
V_IN_CSV_HEADER = [col + ',' for col in V_IN_CSV_HEADER]
# Remove the last commas
V_IN_CSV_HEADER[-1] = V_IN_CSV_HEADER[-1].rstrip(',')

P_OUT_CSV_HEADER = [SCENE_IDX, CITY, TRACK_ID, WAS_TARGET] + P_OUT_X + P_OUT_Y
# Add commas for each element
P_OUT_CSV_HEADER = [col + ',' for col in P_OUT_CSV_HEADER]
# Remove the last commas
P_OUT_CSV_HEADER[-1] = P_OUT_CSV_HEADER[-1].rstrip(',')

V_OUT_CSV_HEADER = [SCENE_IDX, CITY, TRACK_ID, WAS_TARGET] + V_OUT_X + V_OUT_Y
# Add commas for each element
V_OUT_CSV_HEADER = [col + ',' for col in V_OUT_CSV_HEADER]
# Remove the last commas
V_OUT_CSV_HEADER[-1] = V_OUT_CSV_HEADER[-1].rstrip(',')

In [3]:
# Set the training and test paths

# train_path = DUMMY_TRAIN_PATH
# test_path = DUMMY_TEST_PATH

train_path = TRAIN_PATH
test_path = TEST_PATH

CSV_PATH = IN_VEL_FILE_TEST  # target file 
CSV_HEADER = V_IN_CSV_HEADER  # target header

BATCH_SIZE = 4
NUM_WORKERS = 0

In [4]:
class ArgoverseDatasetEDA(torch.utils.data.Dataset):
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDatasetEDA, self).__init__()
        self.data_path = data_path
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):
        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        return data

In [5]:
def collate_train_eda(batch):
    """ 
    Custom collate_fn function for train dataset. 
        """          
    batch_data = []
    for scene in batch:  
        # Get data for tracked agents
        idxs = np.nonzero(scene[CAR_MASK])[0]  # indexes of tracked agents out of the 60
        scene_idxs = [scene[SCENE_IDX]] * len(idxs)
        city = [scene[CITY]] * len(idxs)
        track_ids = scene[TRACK_ID][idxs, 0, 0]  # id's of tracked agents
        was_target = (track_ids == scene[AGENT_ID]).astype(int)  # whether tracked agent was the target
        
        # @ CHANGE these lines accordingly
#         p_in_x = scene[P_IN][idxs, :, 0]  # all p_in x-components
#         p_in_y = scene[P_IN][idxs, :, 1]  # all p_in y-components 
#         batch_data.append([scene_idxs, city, track_ids, was_target, p_in_x, p_in_y])                   


        v_in_x = scene[V_IN][idxs, :, 0]  # all v_in x-components
        v_in_y = scene[V_IN][idxs, :, 1]  # all v_in y-components 
        batch_data.append([scene_idxs, city, track_ids, was_target, v_in_x, v_in_y]) 

    out = []
    return [batch_data, out]

In [6]:
def collate_test_eda(batch):
    """ 
    Custom collate_fn for validation dataset.
    """    
    batch_data = []
    for scene in batch:  
        # Get data for tracked agents
        idxs = np.nonzero(scene[CAR_MASK])[0]  # indexes of tracked agents out of the 60
        scene_idxs = [scene[SCENE_IDX]] * len(idxs)
        city = [scene[CITY]] * len(idxs)
        track_ids = scene[TRACK_ID][idxs, 0, 0]  # id's of tracked agents
        was_target = (track_ids == scene[AGENT_ID]).astype(int)  # whether tracked agent was the target
        
        # @ CHANGE these lines accordingly
#         p_in_x = scene[P_IN][idxs, :, 0]  # all p_in x-components
#         p_in_y = scene[P_IN][idxs, :, 1]  # all p_in y-components 
#         batch_data.append([scene_idxs, city, track_ids, was_target, p_in_x, p_in_y])   
        
        v_in_x = scene[V_IN][idxs, :, 0]  # all v_in x-components
        v_in_y = scene[V_IN][idxs, :, 1]  # all v_in y-components         
        batch_data.append([scene_idxs, city, track_ids, was_target, v_in_x, v_in_y])                   


    out = []
    return [batch_data, out]

In [7]:
# dataset = ArgoverseDatasetEDA(data_path=train_path)
# loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=False, 
#                                      collate_fn=collate_train_eda, num_workers=NUM_WORKERS)
dataset = ArgoverseDatasetEDA(data_path=test_path)
loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=False, 
                                     collate_fn=collate_test_eda, num_workers=NUM_WORKERS)

In [8]:
# # Look at one data sample      
# for i_batch, (data, label) in enumerate(loader):
#     print(type(data))
#     print(type(label))
#     break

In [9]:
def data_to_csv(loader):  
    """
    Extracts hard-coded, specified features from Pickle files into CSV file.
    
    BEFORE calling this method:
    1. You'll need to change the CSV_PATH, CSV_HEADER, and scene[4] and scene[5] keys
    for each feature you investigate.
    
    2. Make sure the collate_fn you use has selected the feature you're investigating.
    """
    with open(CSV_PATH, "w") as csv_file:
        # Clear the csv file before appending data to it
        csv_file.truncate()
        # Write the header to the csv file
        csv_file.writelines(CSV_HEADER + ['\n']) 
        
    iterator = tqdm.notebook.tqdm(loader, total=int(len(loader)))
    
    for batch_idx, (data, label) in enumerate(iterator):
        # Convert data into DataFrame
        for scene in data:
            df = pd.DataFrame(
                {
                    SCENE_IDX: scene[0],
                    CITY: scene[1],
                    TRACK_ID: scene[2],
                    WAS_TARGET: scene[3],                            
                }
            )
            # @ CHANGE these lines accordingly
#             df[P_IN_X] = scene[4]
#             df[P_IN_Y] = scene[5]
            df[V_IN_X] = scene[4]
            df[V_IN_Y] = scene[5]
            # Write data to CSV file
            df.to_csv(CSV_PATH, index=False, header=False, mode='a')     

In [11]:
data_to_csv(loader)

# Lane and lane norm extraction

In [19]:
src_path = './original_train'
# path_mia_x = './mia_lane_x.txt'
# path_pit_x = './pit_lane_x.txt'
# path_mia_y = './mia_lane_y.txt'
# path_pit_y = './pit_lane_y.txt'
# key = LANE

path_mia_x = './mia_lanenorm_x.txt'
path_pit_x = './pit_lanenorm_x.txt'
path_mia_y = './mia_lanenorm_y.txt'
path_pit_y = './pit_lanenorm_y.txt'
key = LANE_NORM

In [20]:
with open(path_mia_x, "w") as mia_x, open(path_pit_x, 'w') as pit_x, open(path_mia_y, 'w') as mia_y, open(path_pit_y, 'w') as pit_y:
    with os.scandir(src_path) as entries:  
        iterator = tqdm.notebook.tqdm(entries, total=205942)        
        for entry in iterator:  
            # Load the  pickle file
            with open(entry, "rb") as file:
                scene = pickle.load(file)
                scene_idx = scene[SCENE_IDX]
                city = scene[CITY]
                xlane = scene[key][:, 0]
                ylane = scene[key][:, 1]

                # Form comma-separated string of all lane values
                sx = ','.join( [str(scene_idx)] + [str(num) for num in xlane] )
                sy = ','.join( [str(scene_idx)] + [str(num) for num in ylane] )                  

                # Write data to file 
                if city == 'MIA':
                    mia_x.write(sx + '\n')
                    mia_y.write(sy + '\n')
                else:
                    pit_x.write(sx + '\n')
                    pit_y.write(sy + '\n')

In [18]:
with open(path_mia_y, 'w') as mia_y, open(path_pit_y, 'w') as pit_y:
    with os.scandir(src_path) as entries:  
        iterator = tqdm.notebook.tqdm(entries, total=205942)        
        for entry in iterator:  
            # Load the  pickle file
            with open(entry, "rb") as file:
                scene = pickle.load(file)
                scene_idx = scene[SCENE_IDX]
                city = scene[CITY]
                ylane = scene[key][:, 1]

                # Form comma-separated string of all lane values
                sy = ','.join( [str(scene_idx)] + [str(num) for num in ylane] )                  

                # Write data to file 
                if city == 'MIA':
                    mia_y.write(sy + '\n')
                else:
                    pit_y.write(sy + '\n')

# Simplified data creation

In [5]:
IN_HEADER = ['x' + str(i // 2 + 1) if i % 2 == 0 else 'y' + str(i // 2 + 1) for i in range(38)]
OUT_HEADER = ['x' + str(i // 2 + 1) if i % 2 == 0 else 'y' + str(i // 2 + 1) for i in range(60)]

In [6]:
IN_HEADER = [SCENE_IDX] + IN_HEADER
IN_HEADER = [col + ',' for col in IN_HEADER]
IN_HEADER[-1] = IN_HEADER[-1].rstrip(',')

OUT_HEADER = [SCENE_IDX] + OUT_HEADER
OUT_HEADER = [col + ',' for col in OUT_HEADER]
OUT_HEADER[-1] = OUT_HEADER[-1].rstrip(',')

In [35]:
# Target agents version
with open('./pin_train.csv', "w") as fpin, open('./pout_train.csv', 'w') as fpout, open('./vin_train.csv', 'w') as fvin, open('./vout_train.csv', 'w') as fvout:
    # Write headers to the files
    fpin.writelines(IN_HEADER + ['\n']) 
    fpout.writelines(OUT_HEADER + ['\n']) 
    fvin.writelines(IN_HEADER + ['\n']) 
    fvout.writelines(OUT_HEADER + ['\n']) 

    with os.scandir('./original_train') as entries:  
        iterator = tqdm.notebook.tqdm(entries, total=205942)        
        for entry in iterator:  
            # Load the  pickle file
            with open(entry, "rb") as file:
                scene = pickle.load(file)
                scene_idx = scene[SCENE_IDX]
                # Get the target agent id
                agent_id = scene[AGENT_ID]        
                # Get the matrix of all agents
                track_id = scene[TRACK_ID]        
                # Get the location of the target agent in the matrix
                idx = np.nonzero(track_id[:, 0] == agent_id)[0][0]
                
                pin, pout, vin, vout = scene[P_IN][idx], scene[P_OUT][idx], scene[V_IN][idx], scene[V_OUT][idx]        
                pin = pin.flatten()
                pout = pout.flatten()
                vin = vin.flatten()
                vout = vout.flatten()

                # Form comma-separated string of all values
                spin = ','.join( [str(scene_idx)] + [str(p) for p in pin] )
                spout = ','.join( [str(scene_idx)] + [str(p) for p in pout] )
                svin = ','.join( [str(scene_idx)] + [str(p) for p in vin] )                  
                svout = ','.join( [str(scene_idx)] + [str(p) for p in vout] )
                                
                # Write data to file 
                fpin.write(spin + '\n')
                fpout.write(spout + '\n')
                fvin.write(svin + '\n')
                fvout.write(svout + '\n')  

In [44]:
# All non-dummy agents version
with open('./pin_trainall.csv', "w") as fpin, open('./pout_trainall.csv', 'w') as fpout, open('./vin_trainall.csv', 'w') as fvin, open('./vout_trainall.csv', 'w') as fvout:
    # Write headers to the files
    fpin.writelines(IN_HEADER + ['\n']) 
    fpout.writelines(OUT_HEADER + ['\n']) 
    fvin.writelines(IN_HEADER + ['\n']) 
    fvout.writelines(OUT_HEADER + ['\n']) 

    with os.scandir('./original_train') as entries:  
        iterator = tqdm.notebook.tqdm(entries, total=205942)        
        for entry in iterator:  
            # Load the  pickle file
            with open(entry, "rb") as file:
                scene = pickle.load(file)
                scene_idx = scene[SCENE_IDX]
                idx = int(np.sum(scene[CAR_MASK]))               
                
                pin, pout, vin, vout = scene[P_IN][:idx], scene[P_OUT][:idx], scene[V_IN][:idx], scene[V_OUT][:idx] 
                for i in range(idx):
                    # Form comma-separated string of all values
                    spin = ','.join( [str(scene_idx)] + [str(p) for p in pin[i].flatten()] )
                    spout = ','.join( [str(scene_idx)] + [str(p) for p in pout[i].flatten()] )
                    svin = ','.join( [str(scene_idx)] + [str(p) for p in vin[i].flatten()] )                  
                    svout = ','.join( [str(scene_idx)] + [str(p) for p in vout[i].flatten()] )
                                
                    # Write data to file 
                    fpin.write(spin + '\n')
                    fpout.write(spout + '\n')
                    fvin.write(svin + '\n')
                    fvout.write(svout + '\n')

In [42]:
# Target agents version
with open('./pin_val.csv', "w") as fpin, open('./pout_val.csv', 'w') as fpout, open('./vin_val.csv', 'w') as fvin, open('./vout_val.csv', 'w') as fvout:
    # Write headers to the files
    fpin.writelines(IN_HEADER + ['\n']) 
    fpout.writelines(OUT_HEADER + ['\n']) 
    fvin.writelines(IN_HEADER + ['\n']) 
    fvout.writelines(OUT_HEADER + ['\n']) 

    with os.scandir('./val_data') as entries:  
        iterator = tqdm.notebook.tqdm(entries, total=3200)        
        for entry in iterator:  
            # Load the  pickle file
            with open(entry, "rb") as file:
                scene = pickle.load(file)
                scene_idx = scene[SCENE_IDX]
                # Get the target agent id
                agent_id = scene[AGENT_ID]        
                # Get the matrix of all agents
                track_id = scene[TRACK_ID]        
                # Get the location of the target agent in the matrix
                idx = np.nonzero(track_id[:, 0] == agent_id)[0][0]
                
                pin, vin= scene[P_IN][idx], scene[V_IN][idx]        
                pin = pin.flatten()
                vin = vin.flatten()

                # Form comma-separated string of all values
                spin = ','.join( [str(scene_idx)] + [str(p) for p in pin] )
                svin = ','.join( [str(scene_idx)] + [str(p) for p in vin] )                  
                                
                # Write data to file 
                fpin.write(spin + '\n')
                fvin.write(svin + '\n')

In [7]:
# Lane extraction
CLS_LANE_IN = 'closest_lanes_in'
CLS_LANE_NORM_IN = 'closest_lane_norms_in'
CLS_LANE_OUT = 'closest_lanes_out'
CLS_LANE_NORM_OUT = 'closest_lane_norms_out'

In [7]:
# All non-dummy agents version
with open('./lanein.csv', "w") as flin, open('./normin.csv', 'w') as fnin, open('./laneout.csv', "w") as flout, open('./normout.csv', "w") as fnout:
    # Write headers to the files
    flin.writelines(IN_HEADER + ['\n']) 
    fnin.writelines(IN_HEADER + ['\n']) 
    flout.writelines(OUT_HEADER + ['n'])
    fnout.writelines(OUT_HEADER + ['n'])

    with os.scandir('./transformed_train_data') as entries:  
        iterator = tqdm.notebook.tqdm(entries, total=205942)        
        for entry in iterator:  
            # Load the  pickle file
            with open(entry, "rb") as file:
                scene = pickle.load(file)
                scene_idx = scene[SCENE_IDX]
                idx = int(np.sum(scene[CAR_MASK]))               
                
                lin, nin, = scene[CLS_LANE_IN][:idx], scene[CLS_LANE_NORM_IN][:idx]
                lout, nout = scene[CLS_LANE_OUT][:idx], scene[CLS_LANE_NORM_OUT][:idx] 
                for i in range(idx):
                    # Form comma-separated string of all values
                    slin = ','.join( [str(scene_idx)] + [str(p) for p in lin[i].flatten()] )
                    snin = ','.join( [str(scene_idx)] + [str(p) for p in nin[i].flatten()] )
                    slout = ','.join( [str(scene_idx)] + [str(p) for p in lout[i].flatten()] )                  
                    snout = ','.join( [str(scene_idx)] + [str(p) for p in nout[i].flatten()] )
                                
                    # Write data to file 
                    flin.write(slin + '\n')
                    fnin.write(snin + '\n')
                    flout.write(slout + '\n')
                    fnout.write(snout + '\n')

In [9]:
# All non-dummy agents version
with open('targ_val_data/lanein.csv', "w") as flin, open('targ_val_data/normin.csv', 'w') as fnin:
    # Write headers to the files
    flin.writelines(IN_HEADER + ['\n']) 
    fnin.writelines(IN_HEADER + ['\n']) 

    with os.scandir('./transformed_val_data') as entries:  
        iterator = tqdm.notebook.tqdm(entries, total=3200)        
        for entry in iterator:  
            # Load the  pickle file
            with open(entry, "rb") as file:
                scene = pickle.load(file)
                scene_idx = scene[SCENE_IDX]
                idx = int(np.sum(scene[CAR_MASK]))               
                
                lin, nin, = scene[CLS_LANE_IN][:idx], scene[CLS_LANE_NORM_IN][:idx]
                for i in range(idx):
                    # Form comma-separated string of all values
                    slin = ','.join( [str(scene_idx)] + [str(p) for p in lin[i].flatten()] )
                    snin = ','.join( [str(scene_idx)] + [str(p) for p in nin[i].flatten()] )
                                
                    # Write data to file 
                    flin.write(slin + '\n')
                    fnin.write(snin + '\n')
